In [ ]:
!pip show trl

In [ ]:
!pip install git+https://github.com/huggingface/trl.git

  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-llzr8j0s
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-llzr8j0s
  Resolved https://github.com/huggingface/trl.git to commit 1123bd0f514164eb297e7d6d48d8d8057c6e7334
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Created wheel for trl: filename=trl-0.14.0.dev0-py3-none-any.whl size=310638 sha256=4b81ccc262301d0b67f1e754a145ea0936b7e0b5a1b4764be869040e26c28392
  Stor

In [ ]:

!python -c "import trl; print(trl.__version__)"



0.14.0.dev0


In [ ]:
!pip install datasets

In [ ]:
import os

import torch

import pandas as pd


import datasets


from tqdm import tqdm

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

#Causal LM - causal language model
from transformers import AutoTokenizer, AutoModelForCausalLM


from huggingface_hub import login

# SFT - Supervised fine tuning
from trl import SFTTrainer, SFTConfig

In [ ]:
# Mount Google Drive for dataset access
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define dataset paths
train_file = '/content/drive/MyDrive/dataScienceLab/train_df_preprocessed.csv'
test_file = '/content/drive/MyDrive/dataScienceLab/test_df_preprocessed.csv'
valid_file = '/content/drive/MyDrive/dataScienceLab/valid_df_preprocessed.csv'

In [ ]:
# Load datasets into pandas DataFrames
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)
valid_df = pd.read_csv(valid_file)

print(train_df.head())
print(train_df.info())


         Label                                          Statement  \
0        FALSE  say annies list political group support third ...   
1    half-true  decline coal start started natural gas took st...   
2  mostly-true  hillary clinton agrees john mccain voting give...   
3        FALSE  health care reform legislation likely mandate ...   
4    half-true               economic turnaround started end term   

                              Subject         Speaker             Job Title  \
0                            abortion    dwayne-bohac  State representative   
1  energy,history,job-accomplishments  scott-surovell        State delegate   
2                      foreign-policy    barack-obama             President   
3                         health-care    blog-posting               Unknown   
4                        economy,jobs   charlie-crist               Unknown   

      State       Party  Barely True Count  False Count  Half True Count  \
0     Texas  republican           

In [ ]:
# Check device availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Converts the pandas DataFrames into Hugging Face Dataset objects,
# which are optimized for processing and training.

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

print(train_dataset)
print(valid_dataset)
print(test_dataset)

Dataset({
    features: ['Label', 'Statement', 'Subject', 'Speaker', 'Job Title', 'State', 'Party', 'Barely True Count', 'False Count', 'Half True Count', 'Mostly True Count', 'Pants on Fire Count', 'Context', 'Party_Categorized', 'Adjusted Credit History', 'Label_encoded', 'False Ratio', 'Sentiment'],
    num_rows: 10263
})
Dataset({
    features: ['Label', 'Statement', 'Subject', 'Speaker', 'Job Title', 'State', 'Party', 'Barely True Count', 'False Count', 'Half True Count', 'Mostly True Count', 'Pants on Fire Count', 'Context', 'Party_Categorized', 'Adjusted Credit History', 'Label_encoded', 'False Ratio', 'Sentiment'],
    num_rows: 1284
})
Dataset({
    features: ['Label', 'Statement', 'Subject', 'Speaker', 'Job Title', 'State', 'Party', 'Barely True Count', 'False Count', 'Half True Count', 'Mostly True Count', 'Pants on Fire Count', 'Context', 'Party_Categorized', 'Adjusted Credit History', 'Label_encoded', 'False Ratio', 'Sentiment'],
    num_rows: 1267
})


In [ ]:
#Template strings used to generate prompts for classification tasks.
##Training Prompt: Includes both the statement and its label (ground truth).
#Inference Prompt: Includes only the statement; the model is expected to predict the label.

TRAINING_CLASSIFIER_PROMPT = """### Statement: {statement} ### Class: {label}"""
INFERENCE_CLASSIFIER_PROMPT = """### Statement: {statement} ### Class:"""


#INFERENCE_CLASSIFIER_PROMPTv2 = """ ### Statement: {statement} ### Class: [Choose only one of: FALSE, half-true, mostly-true, TRUE, barely-true, pants-fire]"""


In [ ]:
def format_data(dataset, prompt, mode="train"):

    formatted_data = dataset.apply(
        lambda row: {
            "instructions": prompt.format(statement=row["Statement"], label=row["Label"] if mode == "train" else ""),
            "label": row["Label"]
        },
        axis=1
    )
    return Dataset.from_pandas(pd.DataFrame(formatted_data.tolist()))


In [ ]:
train_dataset = format_data(train_df, TRAINING_CLASSIFIER_PROMPT, mode="train")
valid_dataset = format_data(valid_df, TRAINING_CLASSIFIER_PROMPT, mode="train")
test_dataset = format_data(test_df, INFERENCE_CLASSIFIER_PROMPT, mode="test")

print(train_dataset)
print(valid_dataset)
print(test_dataset)

Dataset({
    features: ['instructions', 'label'],
    num_rows: 10263
})
Dataset({
    features: ['instructions', 'label'],
    num_rows: 1284
})
Dataset({
    features: ['instructions', 'label'],
    num_rows: 1267
})


In [ ]:
print(test_dataset['label'])

['TRUE', 'FALSE', 'FALSE', 'half-true', 'pants-fire', 'TRUE', 'TRUE', 'barely-true', 'TRUE', 'barely-true', 'barely-true', 'barely-true', 'pants-fire', 'FALSE', 'half-true', 'TRUE', 'pants-fire', 'half-true', 'TRUE', 'FALSE', 'mostly-true', 'half-true', 'pants-fire', 'mostly-true', 'FALSE', 'TRUE', 'barely-true', 'half-true', 'TRUE', 'mostly-true', 'FALSE', 'FALSE', 'half-true', 'half-true', 'TRUE', 'half-true', 'barely-true', 'mostly-true', 'TRUE', 'TRUE', 'barely-true', 'half-true', 'pants-fire', 'barely-true', 'mostly-true', 'barely-true', 'half-true', 'mostly-true', 'FALSE', 'barely-true', 'half-true', 'barely-true', 'mostly-true', 'half-true', 'FALSE', 'half-true', 'mostly-true', 'half-true', 'pants-fire', 'pants-fire', 'barely-true', 'barely-true', 'TRUE', 'FALSE', 'half-true', 'FALSE', 'mostly-true', 'mostly-true', 'TRUE', 'TRUE', 'half-true', 'half-true', 'barely-true', 'mostly-true', 'FALSE', 'half-true', 'FALSE', 'FALSE', 'half-true', 'barely-true', 'pants-fire', 'half-true',

In [ ]:
#Logs into Hugging Face to enable downloading large pre-trained models like LLaMA.
#uncomment and add your token below to login.
# login(token="")

In [ ]:
# Load model and tokenizer
# Loads the LLaMA 2 model and its tokenizer from Hugging Face:
# Tokenizer: Converts text to token IDs.
# Model: LLaMA 2-7B (7 billion parameters), loaded for causal language modeling.

model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, trust_remote_code=True, device_map="auto", use_cache=False
).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
# A padding token set.
# Padding is needed when processing sequences of different lengths to ensure they are all the same size for input to the model.

# Check if the tokenizer has a pad_token, otherwise add one
if tokenizer.pad_token is None:
    # Use the eos_token as the padding token if available
    if tokenizer.eos_token:
        tokenizer.pad_token = tokenizer.eos_token
    else:
        # Add a new pad token if no eos_token exists
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        tokenizer.pad_token = '[PAD]'

# Ensure the model is aware of the new padding token
model.resize_token_embeddings(len(tokenizer))


Embedding(128256, 2048)

In [ ]:
!pip show trl

Name: trl
Version: 0.14.0.dev0
Summary: Train transformer language models with reinforcement learning.
Home-page: https://github.com/huggingface/trl
Author: Leandro von Werra
Author-email: leandro.vonwerra@gmail.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: accelerate, datasets, rich, transformers
Required-by: 


In [ ]:
# Fine-tuning configuration
# Configures parameters for supervised fine-tuning (SFT):

# max_seq_length: Maximum token length for input sequences.

# learning_rate: Speed of model updates during training.

# batch_size: Number of samples processed at once.

# num_train_epochs: Total passes through the training data.

# save_steps and save_strategy: Save the model after each epoch.

sft_config = SFTConfig(

    dataset_text_field='instructions',
    learning_rate=5e-5,
    max_seq_length=256,
    dataset_batch_size = 16,
    num_train_epochs = 2,
    output_dir='/content/drive/MyDrive/dataScienceLab/fine_tuned_model',
    run_name="fine_tuning_llama_321B",
    evaluation_strategy = "steps", # evaluate after fixed number of steps
    eval_steps = 500, # perform evaluation every 500 step
    logging_steps = 500, # log metrics every 500 steps
    save_strategy = "steps", # save after a fixed number of steps
    save_steps = 500,# save a checkpoint every 500 steps
    save_total_limit = 1, # retain only the 2 most recent checkpoints
    load_best_model_at_end = True, # load the best checkpoint at the end
    metric_for_best_model = "eval_loss", # use evaluation loss to determine the best model
    greater_is_better = False # lower eval_loss is better

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Fine-tuning using SFTTrainer (SUPERVISED FINE TUNING)

# SFTTrainer: A tool for fine-tuning pre-trained language models.
# Training:
# Uses the formatted training and validation datasets.
# Automatically handles training, validation, and checkpoint saving.

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    processing_class=tokenizer,
    args=sft_config,

)

# Start training
trainer.train()

Map:   0%|          | 0/10263 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,3.501700,3.509774
1000,3.256800,3.330977
1500,2.670900,3.475003
2000,1.944000,3.450840
2500,1.846500,3.442775


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=2566, training_loss=2.6233285887577713, metrics={'train_runtime': 1006.505, 'train_samples_per_second': 20.393, 'train_steps_per_second': 2.549, 'total_flos': 3843358825070592.0, 'train_loss': 2.6233285887577713, 'epoch': 2.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/dataScienceLab/fine_tuned_model/final_model")
tokenizer.save_pretrained("/content/drive/MyDrive/dataScienceLab/fine_tuned_model/final_model")


('/content/drive/MyDrive/dataScienceLab/fine_tuned_model/final_model/tokenizer_config.json',
 '/content/drive/MyDrive/dataScienceLab/fine_tuned_model/final_model/special_tokens_map.json',
 '/content/drive/MyDrive/dataScienceLab/fine_tuned_model/final_model/tokenizer.json')

In [ ]:
!nvidia-smi

Sun Jan 26 09:57:15 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              50W / 400W |  25361MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
# load the saved best model

# Path to the directory where the model is saved
output_dir = "/content/drive/MyDrive/dataScienceLab/fine_tuned_model/final_model"

# Load the tokenizer and model
tokenizer_for_tuned_model = AutoTokenizer.from_pretrained(output_dir)
tuned_model = AutoModelForCausalLM.from_pretrained(output_dir).to(device)

In [ ]:
!ls /content/drive/MyDrive/dataScienceLab/fine_tuned_model

checkpoint-1200  checkpoint-1283  runs


In [ ]:

from sklearn.metrics import classification_report
import re
from collections import Counter

In [ ]:
# classes for prediction
valid_classes = ["FALSE", "half-true", "mostly-true", "TRUE", "barely-true", "pants-fire"]

In [ ]:
def extract_label(model_output, valid_classes):
    """
    Extracts the label from the model output, accounting for multiple class labels and
    choosing the majority class from a predefined list of valid classes.

    Args:
    - model_output (str): The model's output string.
    - valid_classes (list): A list of valid class labels.

    Returns:
    - str: The predicted class label based on majority vote, or 'unknown' if no valid class is found.
    """
    # Define the pattern to capture the part after "### Class:" and match the valid class labels
    pattern = r"### Class:\s*(.*)"

    # Search for the pattern in the model output
    match = re.search(pattern, model_output)

    if match:
        # Get the part after "### Class:"
        class_str = match.group(1)

        # Split the string into individual class labels (by space)
        class_labels = class_str.split()

        # Filter valid class labels
        valid_class_labels = [label for label in class_labels if label in valid_classes]

        if valid_class_labels:
            # Count occurrences of each valid class label
            class_counts = Counter(valid_class_labels)

            # Get the most common (majority) class
            majority_class, _ = class_counts.most_common(1)[0]
            return majority_class
        else:
            # Return "unknown" if no valid class is found
            return "unknown"
    else:
        # Return "unknown" if no "### Class:" is found in the model output
        return "unknown"

In [ ]:


# Evaluation function
def run_eval(dataset, model, tokenizer, limit_test_size=10):

    instructions, labels = dataset["instructions"], dataset["label"]


    print(f" instructions from dataset {instructions}")
    print(f" labels from dataset {labels}")

    predictions = []

    for instruct, label in tqdm(zip(instructions[:limit_test_size], labels[:limit_test_size])):
        print(f'Instruction: {instruct}')
        print(f'Ground Truth Label: {label}')

        input_ids = tokenizer(instruct, return_tensors="pt", truncation=True).input_ids.to(device)
        attention_mask = tokenizer(instruct, return_tensors="pt", truncation=True, padding=True, return_attention_mask=True).attention_mask.to(device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_new_tokens=10,
                do_sample=False,
                top_p=1.,
                temperature=0.0,
                #return_dict_in_generate=True,
                #output_scores=True
            )

        # Decode model output
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f'Model Output: {result}')

        # Extract the predicted label using regex or simple string manipulation
        # Assuming the label is appended after "### Class:"
        prediction = extract_label(result, valid_classes)
        print(f'Extracted Prediction: {prediction}')
        predictions.append(prediction)

    # Generate the classification report
    report = classification_report(labels[:limit_test_size], predictions, zero_division=0)
    print("Full Classification Report:\n", report)


In [ ]:
# Runs evaluation function on the test dataset.

# full size is 1267
limit_test_size = 1267

run_eval(test_dataset, tuned_model, tokenizer_for_tuned_model, limit_test_size)

 instructions from dataset ['### Statement: building wall u mexico border take literally year ### Class:', '### Statement: wisconsin pace double number layoff year ### Class:', '### Statement: say john mccain done nothing help vet ### Class:', '### Statement: suzanne bonamici support plan cut choice medicare advantage senior ### Class:', '### Statement: asked reporter whether he center criminal scheme violate campaign law gov scott walker nodded yes ### Class:', '### Statement: past five year federal government paid 601 million retirement disability benefit deceased former federal employee ### Class:', '### Statement: say tennessee law requires school receive half proceeds 31 million per year half cent increase shelby county sale tax ### Class:', '### Statement: say vice president joe biden admits american people scammed economic stimulus package ### Class:', '### Statement: donald trump marriage equality want go back ### Class:', '### Statement: know half hillary clinton meeting secre

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: ### Statement: building wall u mexico border take literally year ### Class:
Ground Truth Label: TRUE


1it [00:00,  4.49it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Model Output: ### Statement: building wall u mexico border take literally year ### Class: FALSE ### Class: FALSE half-true ### Class
Extracted Prediction: FALSE
Instruction: ### Statement: wisconsin pace double number layoff year ### Class:
Ground Truth Label: FALSE


...

Model Output: ### Statement: early 1980s sen edward kennedy secretly offered help soviet leader counter reagan administration position nuclear disarmament ### Class: FALSE ### Class: FALSE half-true ### Class
Extracted Prediction: FALSE
Instruction: ### Statement: say epa permit languished strickland new epa director got done two day ### Class:
Ground Truth Label: barely-true
Model Output: ### Statement: say epa permit languished strickland new epa director got done two day ### Class: FALSE FALSE FALSE ### Class: FALSE FALSE FALSE ###

1266it [04:21,  4.87it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Extracted Prediction: FALSE
Instruction: ### Statement: say governor going around state talking fund income tax cut benefit higher income earner lower income earner ### Class:
Ground Truth Label: FALSE


1267it [04:21,  4.84it/s]

Model Output: ### Statement: say governor going around state talking fund income tax cut benefit higher income earner lower income earner ### Class: FALSE ### Class: FALSE half-true ### Class
Extracted Prediction: FALSE
Full Classification Report:
               precision    recall  f1-score   support

       FALSE       0.23      0.81      0.36       249
        TRUE       0.26      0.12      0.17       208
 barely-true       0.50      0.00      0.01       212
   half-true       0.38      0.07      0.12       265
 mostly-true       0.29      0.30      0.30       241
  pants-fire       0.33      0.01      0.02        92

    accuracy                           0.25      1267
   macro avg       0.33      0.22      0.16      1267
weighted avg       0.33      0.25      0.18      1267

